In [ ]:
import dotenv

Импортируем себе данные из БД (на примере SQLite)

In [ ]:
import sqlite3

def get_data_from_db(db_name="your_database.db"):
    """
    Получение данных из базы данных с добавлением информации о тематике (category)
    """
    conn = sqlite3.connect(db_name)
    cursor = conn.cursor()
    cursor.execute("SELECT id, text, category FROM documents")  # Включаем столбец 'category'
    data = cursor.fetchall()
    conn.close()
    return data

Подготовка данных к векторизации

In [ ]:
import re

# Пример списка ругательных слов
bad_words = {"глупый", "тупой", "идиот", "дурак", "пошел вон", "сука"}  # Можно расширить

# Функция для препроцессинга текста
def preprocess_text(text):
    # Приводим текст к нижнему регистру
    text = text.lower()

    # Удаляем ругательные слова
    text = " ".join([word for word in text.split() if word not in bad_words])

    # Удаляем пунктуацию
    text = re.sub(r'[^\w\s]', '', text)

    # Удаляем лишние пробелы
    text = re.sub(r'\s+', ' ', text).strip()

    return text

In [ ]:
metadata.append({
    "document_id": doc_id,
    "chunk_idx": chunk_idx,
    "text": chunk,
    "full_text_length": len(text),
    "chunk_text_length": len(chunk),
    "category": category  # Добавляем категорию
})

Векторизация

In [ ]:
import requests

def vectorize_with_gigachat(text):
    response = requests.post("http://localhost:5000/v1/encode", json={"text": text})
    vector = response.json().get('vector')  # Пример получения вектора из ответа
    return vector

индексация с помощью FAISS

In [ ]:
import faiss
import numpy as np

def build_faiss_index(vectors):
    dimension = len(vectors[0])  # Должно соответствовать размерности вектора
    index = faiss.IndexFlatL2(dimension)  # Индекс по L2-метрике

    # Преобразование векторов в массив numpy
    vectors_np = np.array(vectors).astype(np.float32)
    
    # Добавление векторов в индекс
    index.add(vectors_np)
    return index

Нужно развернуть qdrant в докере, чтобы можно было потестить локально
docker pull qdrant/qdrant
docker run -p 6333:6333 qdrant/qdrant

Запись в qdrant

In [ ]:
from qdrant_client import QdrantClient
from qdrant_client.models import VectorParams, Distance, PointStruct

def write_to_qdrant(vectors, metadata):
    client = QdrantClient(host="localhost", port=6333)  # Порт по умолчанию
    collection_name = "documents_collection"

    # Преобразование векторов в точки
    points = [PointStruct(id=i, vector=vec, payload={"metadata": meta}) for i, (vec, meta) in enumerate(zip(vectors, metadata))]

    # Создание коллекции и запись точек
    client.upsert(collection_name=collection_name, points=points)

реализация поиска

def search_in_qdrant(query_vector):
    client = QdrantClient(host="localhost", port=6333)
    collection_name = "documents_collection"

    # Запрос к Qdrant для поиска по вектору
    search_result = client.search(
        collection_name=collection_name,
        query_vector=query_vector,
        top=5  # Топ-5 наиболее похожих результатов
    )
    return search_result

Генерация овета с OpenAI

In [ ]:
def generate_answer(prompt):
    openai.api_key = "your-openai-api-key"
    response = openai.Completion.create(
        model="gpt-4",
        prompt=prompt,
        max_tokens=100
    )
    return response.choices[0].text.strip()

собираем вместе

In [ ]:
def rag_system_with_preprocessing(query):
    try:
        # 1. Получение данных из базы данных
        db_data = get_data_from_db()

        vectors = []
        metadata = []
        all_chunks = []  # Сюда будем собирать все чанки с метаданными

        for doc_id, text, category in db_data:  # Теперь получаем 'category'
            try:
                # 2. Применяем препроцессинг к тексту
                processed_text = preprocess_text(text)

                # 3. Разбиваем текст на чанки
                chunks = split_text_into_chunks(processed_text)

                for chunk_idx, chunk in enumerate(chunks):
                    try:
                        # 4. Векторизация каждого чанка
                        vector = vectorize_with_gigachat(chunk)
                        vectors.append(vector)

                        # 5. Добавление метаданных
                        metadata.append({
                            "document_id": doc_id,
                            "chunk_idx": chunk_idx,
                            "text": chunk,
                            "full_text_length": len(text),
                            "chunk_text_length": len(chunk),
                            "category": category  # Добавляем категорию в метаданные
                        })
                        all_chunks.append(chunk)

                    except Exception as e:
                        print(f"Ошибка в векторизации чанка {chunk_idx} документа {doc_id}: {e}")

            except Exception as e:
                print(f"Ошибка в обработке документа {doc_id}: {e}")

        # 6. Индексация в FAISS
        faiss_index = build_faiss_index(vectors)

        # 7. Запись в Qdrant с добавлением метаданных
        write_to_qdrant(vectors, metadata)

        # 8. Применяем препроцессинг к запросу пользователя
        processed_query = preprocess_text(query)

        # 9. Векторизация запроса пользователя
        query_vector = vectorize_with_gigachat(processed_query)

        # 10. Поиск в Qdrant, ограничиваем результат до 3
        search_results = search_in_qdrant(query_vector, top=3)

        # 11. Генерация ответа через OpenAI
        top_result = search_results[0]  # Лучший результат
        response_text = top_result.payload['metadata']['text']

        prompt = f"На основе следующего контекста: {response_text}\nОтветьте на следующий запрос: {query}"
        answer = generate_answer(prompt)

        # Вывод ответа
        print(answer)

    except Exception as e:
        print(f"Ошибка в основной системе: {e}")

# Пример использования
query = "Какие новые технологии использованы в нашей системе?"
rag_system_with_preprocessing(query)